In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Mesothelioma"
cohort = "GSE68950"

# Input paths
in_trait_dir = "../../input/GEO/Mesothelioma"
in_cohort_dir = "../../input/GEO/Mesothelioma/GSE68950"

# Output paths
out_data_file = "../../output/preprocess/Mesothelioma/GSE68950.csv"
out_gene_data_file = "../../output/preprocess/Mesothelioma/gene_data/GSE68950.csv"
out_clinical_data_file = "../../output/preprocess/Mesothelioma/clinical_data/GSE68950.csv"
json_path = "../../output/preprocess/Mesothelioma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE68950_family.soft.gz', 'GSE68950_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Mesothelioma/GSE68950/GSE68950_family.soft.gz
Matrix file: ../../input/GEO/Mesothelioma/GSE68950/GSE68950_series_matrix.txt.gz


Background Information:
!Series_title	"caArray_golub-00327: Sanger cell line Affymetrix gene expression project"
!Series_summary	"The microarray gene expression pattern was studied using 798 different cancer cell lines. The cancer cell lines are obtained from different centers. Annotation information were provided in the supplementary file."
!Series_overall_design	"golub-00327"
!Series_overall_design	"Assay Type: Gene Expression"
!Series_overall_design	"Provider: Affymetrix"
!Series_overall_design	"Array Designs: HT_HG-U133A"
!Series_overall_design	"Organism: Homo sapiens (ncbitax)"
!Series_overall_design	"Tissue Sites: leukemia, Urinary tract, Lung, BiliaryTract, Autonomic Ganglion, Thyroid gland, Stomach, Breast, Pancreas, Head and Neck, Lymphoma, Colorectal, Placenta, Liver, Brain, Bone, pleura, Skin, endometrium, Ovary, cervix, Oesophagus, Connective and Soft Tissue, Muscle, Kidney, Prostate, Adrenal Gland, Eye, Testis, Smooth Muscle Tissue, Vulva, Unknow"
!Series_overall_design	"M

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Dict, Any, Optional, List

# 1. Gene Expression Data Availability
# From the background information, this is an Affymetrix gene expression project using HT_HG-U133A arrays
# This indicates that gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Let's check for trait (Mesothelioma) availability
# From the background information, Mesothelioma is explicitly listed as one of the disease states
# Looking at disease state in row 1 of the sample characteristics
trait_row = 1  # 'disease state' is in row 1

# Define conversion function for trait
def convert_trait(val):
    if val is None:
        return None
    
    if ":" in val:
        val = val.split(":", 1)[1].strip().lower()
    else:
        val = val.lower()
    
    # Binary classification: 1 for Mesothelioma, 0 for other diseases
    if "mesothelioma" in val:
        return 1
    else:
        return 0

# No age information is available in the sample characteristics
age_row = None

# No gender information is available in the sample characteristics
gender_row = None

# These functions won't be used since the data is not available, but defining them for completeness
def convert_age(val):
    if val is None:
        return None
    
    if ":" in val:
        val = val.split(":", 1)[1].strip()
    
    try:
        return float(val)
    except (ValueError, TypeError):
        return None

def convert_gender(val):
    if val is None:
        return None
    
    if ":" in val:
        val = val.split(":", 1)[1].strip().lower()
    else:
        val = val.lower()
    
    if "female" in val or "f" == val:
        return 0
    elif "male" in val or "m" == val:
        return 1
    else:
        return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                              is_gene_available=is_gene_available, 
                              is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features using sample characteristics data from previous step
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,  # Using clinical_data from previous step
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the extracted clinical data
    preview = preview_df(selected_clinical_df)
    print("Clinical data preview:", preview)
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)


Clinical data preview: {'GSM1687570': [0.0], 'GSM1687571': [0.0], 'GSM1687572': [0.0], 'GSM1687573': [0.0], 'GSM1687574': [0.0], 'GSM1687575': [0.0], 'GSM1687576': [0.0], 'GSM1687577': [0.0], 'GSM1687578': [0.0], 'GSM1687579': [0.0], 'GSM1687580': [0.0], 'GSM1687581': [0.0], 'GSM1687582': [0.0], 'GSM1687583': [0.0], 'GSM1687584': [0.0], 'GSM1687585': [0.0], 'GSM1687586': [0.0], 'GSM1687587': [0.0], 'GSM1687588': [0.0], 'GSM1687589': [0.0], 'GSM1687590': [0.0], 'GSM1687591': [0.0], 'GSM1687592': [0.0], 'GSM1687593': [0.0], 'GSM1687594': [0.0], 'GSM1687595': [0.0], 'GSM1687596': [0.0], 'GSM1687597': [0.0], 'GSM1687598': [0.0], 'GSM1687599': [0.0], 'GSM1687600': [0.0], 'GSM1687601': [0.0], 'GSM1687602': [0.0], 'GSM1687603': [0.0], 'GSM1687604': [0.0], 'GSM1687605': [0.0], 'GSM1687606': [0.0], 'GSM1687607': [0.0], 'GSM1687608': [0.0], 'GSM1687609': [0.0], 'GSM1687610': [0.0], 'GSM1687611': [0.0], 'GSM1687612': [0.0], 'GSM1687613': [0.0], 'GSM1687614': [0.0], 'GSM1687615': [0.0], 'GSM168761

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 22277
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examine the gene identifiers in the gene expression data
# These appear to be probe IDs from an Affymetrix microarray (e.g., '1007_s_at', '1053_at')
# They are not standard human gene symbols and will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': [nan, nan, nan, nan, nan], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEFINITION=HSPAX8A H.sapiens Pax8 mRNA', 'L36861 /FEATURE=expanded_cds /DEFINITION=HUMGCAPB Homo sapiens 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Analyze gene identifiers and data structure
# From looking at the data, these are the mappings:
# - Gene identifiers in the gene expression data are stored under 'ID' in the gene annotation dataframe
# - Gene symbols are stored under 'Gene Symbol' in the gene annotation dataframe

# 2. Get gene mapping dataframe
prob_col = 'ID'
gene_col = 'Gene Symbol'
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

# Extract the gene expression data first
expression_df = get_genetic_data(matrix_file)

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(expression_df, mapping_df)

# Print the first few rows of the mapped gene data
print("First 10 genes after mapping:")
print(gene_data.index[:10])
print("Number of genes after mapping:", len(gene_data))


First 10 genes after mapping:
Index(['A2BP1', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAK1',
       'AAMP', 'AANAT'],
      dtype='object', name='Gene')
Number of genes after mapping: 13046


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print(f"Top 10 gene indices before normalization: {gene_data.index[:10].tolist()}")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Top 10 gene indices after normalization: {normalized_gene_data.index[:10].tolist()}")
print(f"Shape of normalized gene data: {normalized_gene_data.shape}")

# Create directory for gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
# Save the normalized gene data
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Extract clinical features using the clinical data from step 1
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Extract clinical features using the convert_trait function from step 2
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=1,  # From step 2
    convert_trait=convert_trait,
    age_row=None,
    convert_age=None,
    gender_row=None,
    convert_gender=None
)

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Saved clinical data to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data.shape}")

# 5. Determine if the trait and demographic features are biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Validate the dataset and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_trait_biased,
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from juvenile myositis (JM) and childhood-onset lupus (cSLE) skin biopsies."
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed. Final linked data not saved.")

Top 10 gene indices before normalization: ['A2BP1', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAK1', 'AAMP', 'AANAT']
Top 10 gene indices after normalization: ['A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAK1', 'AAMDC', 'AAMP', 'AANAT']
Shape of normalized gene data: (12700, 798)


Saved normalized gene data to ../../output/preprocess/Mesothelioma/gene_data/GSE68950.csv


Saved clinical data to ../../output/preprocess/Mesothelioma/clinical_data/GSE68950.csv
Shape of linked data: (798, 12701)


Shape of linked data after handling missing values: (798, 12701)
For the feature 'Mesothelioma', the least common label is '1.0' with 6 occurrences. This represents 0.75% of the dataset.
The distribution of the feature 'Mesothelioma' in this dataset is fine.



Saved processed linked data to ../../output/preprocess/Mesothelioma/GSE68950.csv
